# Genetic Algorithm for Substitution Cipher Decoding
## 1. Problem Overview
A **Substitution Cipher** replaces each letter of the alphabet with another letter. For the English alphabet, there are $26!$ (approximately $4 \times 10^{26}$) possible keys, making a brute-force search impossible. 

In this project, we use a **Genetic Algorithm (GA)** to evolve a population of potential keys (chromosomes). The "fitness" of each key is determined by how much "sense" the decoded text makes, which we evaluate using a pre-built dictionary from a large corpus of English text.

## 2. Dataset Preparation and Cleaning
To evaluate a decoded text, we need a reference dictionary. We use `global_text.txt` to extract valid English words. The cleaning pipeline involves:
- Lowercasing all text.
- Removing non-alphabetic characters (punctuation and numbers).
- Filtering out very short words or common stop words (optional but helpful).

In [1]:
import re
import string
import random

def clean_text(text):
    """
    Normalizes text by lowercasing and removing non-alphabetic characters.
    """
    text = text.lower()
    # Replace anything that is not a-z or space with a space
    text = re.sub(r'[^a-z\s]', ' ', text)
    # Remove extra whitespaces
    text = " ".join(text.split())
    return text

# Load the corpus to build the dictionary
with open('global_text.txt', 'r', encoding='utf-8') as f:
    raw_corpus = f.read()

cleaned_corpus = clean_text(raw_corpus)
# Create a set of unique words for O(1) lookup
dictionary = set(cleaned_corpus.split())

print(f"Dictionary built with {len(dictionary)} unique words.")

Dictionary built with 1827 unique words.


## 3. Chromosome Representation
A **Chromosome** is represented as a list of 26 unique characters, where the index corresponds to the alphabet (0='a', 1='b', ..., 25='z'). 
For example, if the chromosome is `['d', 'a', 'c', ...]`, it means:
- 'a' in encoded text -> 'd' in decoded text
- 'b' in encoded text -> 'a' in decoded text

In [2]:
class Chromosome:
    def __init__(self, key=None):
        self.alphabet = list(string.ascii_lowercase)
        if key is None:
            # Random initial key
            self.key = list(string.ascii_lowercase)
            random.shuffle(self.key)
        else:
            self.key = key
        self.fitness = 0

    def decode(self, encoded_text):
        """
        Decodes the text using this chromosome's key.
        """
        # Create a translation table for speed
        table = str.maketrans(string.ascii_lowercase, "".join(self.key))
        # Keep original case and punctuation by only translating lowercase
        return encoded_text.lower().translate(table)

def initialize_population(pop_size):
    return [Chromosome() for _ in range(pop_size)]

# Example check
example_pop = initialize_population(10)
print(f"Generated {len(example_pop)} random chromosomes.")

Generated 10 random chromosomes.


## 4. Fitness Evaluation
The fitness function measures the quality of a chromosome. We decode the target text using the chromosome's key and count how many words in the resulting text exist in our dictionary.
Higher word matches = Higher fitness.

In [3]:
def calculate_fitness(chromosome, encoded_text, dictionary):
    decoded_text = chromosome.decode(encoded_text)
    words = decoded_text.split()
    if not words:
        return 0
    
    match_count = 0
    for word in words:
        if word in dictionary:
            # We can weight longer words more heavily
            match_count += len(word) ** 2 
            
    chromosome.fitness = match_count
    return match_count

# Load the actual text we want to decode
with open('encoded_text.txt', 'r', encoding='utf-8') as f:
    target_encoded_text = f.read()

# Test fitness for one random chromosome
test_chrom = Chromosome()
score = calculate_fitness(test_chrom, target_encoded_text, dictionary)
print(f"Random Chromosome Fitness Score: {score}")

Random Chromosome Fitness Score: 68


## 5. Decoder Class Infrastructure
Following the project requirements, we encapsulate the logic into a `Decoder` class. 
The `decode()` method will eventually contain the full GA loop (Selection, Crossover, Mutation).

In [4]:
class Decoder:
    def __init__(self, encoded_text):
        self.encoded_text = encoded_text
        self.clean_encoded = clean_text(encoded_text)
        # Pre-built dictionary (should be loaded or passed)
        self.dictionary = dictionary 
        
    def decode(self):
        """
        Main Genetic Algorithm Loop.
        Returns the best decoded string found.
        """
        # Placeholder for GA implementation
        # 1. Initialize Population
        # 2. While not converged:
        #    a. Calculate Fitness
        #    b. Selection
        #    c. Crossover & Mutation
        
        # For now, returns a dummy placeholder or random try
        best_candidate = Chromosome()
        return best_candidate.decode(self.encoded_text)

# Testing the structure
d = Decoder(target_encoded_text)
print("Decoder initialized. Ready for GA implementation.")

Decoder initialized. Ready for GA implementation.


## 6. Genetic Operators (Permutation-Safe)
Standard crossover methods can create duplicate letters in a key. To maintain a valid permutation of the 26-letter alphabet, we implement:
- **Ordered Crossover (OX):** Inherits a segment from one parent and fills the rest from the second parent while preserving order and avoiding duplicates.
- **Swap Mutation:** Randomly swaps two letters in the key to introduce genetic diversity.

In [5]:
def ordered_crossover(parent1, parent2):
    """
    Performs Ordered Crossover (OX) on two parent chromosomes.
    """
    size = len(parent1.key)
    # Select two random points for the segment
    start, end = sorted(random.sample(range(size), 2))
    
    # Child 1 segment from Parent 1
    child1_key = [None] * size
    child1_key[start:end] = parent1.key[start:end]
    
    # Fill remaining from Parent 2
    p2_idx = 0
    for i in range(size):
        if child1_key[i] is None:
            while parent2.key[p2_idx] in child1_key:
                p2_idx += 1
            child1_key[i] = parent2.key[p2_idx]
            
    # Repeat for Child 2 (segment from Parent 2, fill from Parent 1)
    child2_key = [None] * size
    child2_key[start:end] = parent2.key[start:end]
    
    p1_idx = 0
    for i in range(size):
        if child2_key[i] is None:
            while parent1.key[p1_idx] in child2_key:
                p1_idx += 1
            child2_key[i] = parent1.key[p1_idx]
            
    return Chromosome(child1_key), Chromosome(child2_key)

def swap_mutation(chromosome, mutation_rate=0.1):
    """
    Randomly swaps two genes (letters) in the key based on mutation_rate.
    """
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(chromosome.key)), 2)
        chromosome.key[idx1], chromosome.key[idx2] = chromosome.key[idx2], chromosome.key[idx1]

## 7. Synthetic Integration Test
This test verifies all implemented components:
1. Creating a small dummy dictionary.
2. Creating an encoded string with a known key.
3. Testing if a "correct" chromosome gets a higher score than a random one.
4. Testing crossover and mutation.

In [6]:
# 1. Setup small environment
test_dict = {"hello", "world", "this", "is", "genetic", "algorithm"}
test_encoded = "itssg ksgit" # Let's say it means "hello world" with a specific key

# 2. Create a "Perfect" Chromosome for this specific mapping
# Let's assume 'i'->'h', 't'->'e', 's'->'l', 'g'->'o', 'k'->'w', 'r'->'d'
perfect_key = list(string.ascii_lowercase)
# Manual mapping for "hello world"
mapping = {'i':'h', 't':'e', 's':'l', 'g':'o', 'k':'w'}
for enc, dec in mapping.items():
    idx = string.ascii_lowercase.index(enc)
    perfect_key[idx] = dec

perfect_chrom = Chromosome(perfect_key)
random_chrom = Chromosome()

# 3. Compare Fitness
perfect_score = calculate_fitness(perfect_chrom, test_encoded, test_dict)
random_score = calculate_fitness(random_chrom, test_encoded, test_dict)

print(f"Decoded with Perfect Key: '{perfect_chrom.decode(test_encoded)}'")
print(f"Perfect Key Score: {perfect_score}")
print(f"Random Key Score: {random_score}")

# 4. Test Crossover
p1, p2 = Chromosome(), Chromosome()
c1, c2 = ordered_crossover(p1, p2)
print(f"\nCrossover check: Child1 is valid permutation? {len(set(c1.key)) == 26}")

# 5. Test Mutation
before_mut = list(c1.key)
swap_mutation(c1, mutation_rate=1.0) # Force mutation
after_mut = c1.key
print(f"Mutation check: Key changed? {before_mut != after_mut}")

Decoded with Perfect Key: 'hello wlohe'
Perfect Key Score: 25
Random Key Score: 0

Crossover check: Child1 is valid permutation? True
Mutation check: Key changed? True
